In [1]:
import gensim
from gensim.test.utils import common_texts
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from gensim.test.utils import get_tmpfile
from gensim.corpora import Dictionary

from gensim.models import doc2vec
from collections import namedtuple
from gensim.test.utils import get_tmpfile

import pandas as pd

from tqdm import tqdm

from konlpy.tag import Twitter
twitter = Twitter()

---
## 서양화 크롤링

In [2]:
import requests
from bs4 import BeautifulSoup 
from tqdm import tqdm
import re
import pandas as pd
import time

headers = {'User-agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/68.0.3440.106 Safari/537.36'}

def get_links(categoryID, page):
    url ='https://terms.naver.com/list.nhn?'
    
    params = {
        'cid':46702, 
        'categoryId':categoryID, # 한국화 46739, 동양화 46740, 서양화 46741
        'page':page
        }
    
    resp = requests.get(url,params=params,headers=headers)
    soup = BeautifulSoup(resp.content,'lxml')
    aList = soup.select('div.info_area > strong > a')
    
    return (each['href'] for each in aList)

def get_info(page_url):
    url = 'https://terms.naver.com/'+page_url
    resp = requests.get(url,headers=headers)
    soup = BeautifulSoup(resp.content,'lxml')
    
    try :
        # 제목
        title = soup.select_one('div.headword_title > h2').getText()
    except Exception :
        title = ""    
        
    try :
        # 아티스트
        artist = soup.select_one('div.headword_title > p.desc').getText()
    except Exception :
        artist = ""   

    try :
        # 작품
        art = soup.select_one('#innerImage0')['data-src']
    except Exception :
        art = ""   


    try :
        # 텍스트
        text= soup.select_one('#size_ct > p').getText()
    except Exception :
        text = ""   
    
    
    # 종류, 기법
    
    art_type = ""
    tech = ""
    movement = ""

    tmp = soup.select('div.wr_tmp_profile > div > table > tbody > tr > th ')
    
    for num in range(len(tmp)) :
        try : 
            if tmp[num].getText() == '종류' :
                art_type = soup.select_one('div.wr_tmp_profile > div > table > tbody > tr:nth-of-type({node_num}) > td'.format(node_num = num+1)).getText()
            elif tmp[num].getText() == '기법' :
                tech = soup.select_one('div.wr_tmp_profile > div > table > tbody > tr:nth-of-type({node_num}) > td'.format(node_num = num+1)).getText()
            elif tmp[num].getText() == '사조' : # ~~ 주의, 주로 서양화에 있음
                movement = soup.select_one('div.wr_tmp_profile > div > table > tbody > tr:nth-of-type({node_num}) > td'.format(node_num = num+1)).getText()
        except Exception as e:
            print(e)
            continue


    return (title, artist, art_type, tech, movement, art, text)

In [3]:
# 서양화 URL을 얻는다.

western_arts_url = []
for page in tqdm(range(1,201)) :
    western_arts_url.extend(get_links(46741, page)) # 한국화 46739, 동양화 46740, 서양화 46741

100%|██████████| 200/200 [04:45<00:00,  1.43s/it]


In [4]:
western_arts = []
for each_url in tqdm(western_arts_url) :
    western_arts.append(get_info(each_url))

100%|██████████| 3000/3000 [1:06:06<00:00,  1.32s/it]


In [5]:
western_data = pd.DataFrame(western_arts,columns=['title','artist','art_type','tech','movement','image','text'])
western_data.to_csv('./western_3000')
western_data.head(100)

,title,artist,art_type,tech,movement,image,text
0,별이 빛나는 밤,빈센트 반 고흐,유화,캔버스에 유채(Oil on canvas),후기인상주의,https://dthumb-phinf.pstatic.net/?src=%22https...,빈센트 반 고흐의 대표작 중 하나로 꼽히는 <별이 빛나는 밤>은 그가 고갱과 다툰 ...
1,아를르의 포룸 광장의 카페 테라스,빈센트 반 고흐,유화,캔버스에 유채(Oil on canvas),후기인상주의,https://dthumb-phinf.pstatic.net/?src=%22https...,"산업 사회로 이행해 가던 19세기 말, 정신적 가치에 대한 열망은 예술가들이 좀 더..."
2,연인(키스),구스타프 클림트,유화,캔버스에 유채(Oil on canvas),상징주의,https://dthumb-phinf.pstatic.net/?src=%22https...,꽃이 흩뿌려진 작은 초원 위에 서 있는 두 연인은 주변과 분리되어 그들을 마치 후광...
3,해바라기,빈센트 반 고흐,유화,캔버스에 유채(Oil on canvas),후기인상주의,https://dthumb-phinf.pstatic.net/?src=%22https...,"1888년, 반 고흐(Vincent Van Gogh, 1853-1890)는 프랑스 ..."
4,아를의 별이 빛나는 밤,빈센트 반 고흐,유화,캔버스에 유채(Huile sur toile),후기인상주의,https://dthumb-phinf.pstatic.net/?src=%22https...,"빈센트 반 고흐가 주고 받았던 서신들을 살펴보면, 그가 얼마나 ‘밤의 효과’에 매료..."
5,에펠탑의 신랑신부,마르크 샤갈,유화,캔버스에 유채(Huile sur toile),,https://dthumb-phinf.pstatic.net/?src=%22https...,말년의 마르크 샤갈이 거주한 라콜린 저택 응접실의 벽난로 위로 그림 한 점이 걸려있...
6,아를의 반 고흐의 방,빈센트 반 고흐,유화,캔버스에 유채(Oil on canvas),후기인상주의,https://dthumb-phinf.pstatic.net/?src=%22https...,반 고흐는 《반 고흐의 방》이라는 이름으로 세 작품을 그렸다. 가장 오래된 버전은 ...
7,극동에서 온 꽃,조영남,유화(회화),혼합재료(Mixed midea),,https://dthumb-phinf.pstatic.net/?src=%22https...,
8,행복한 눈물,로이 릭텐스타인,유화,캔버스에 마그나펜(Magna on canvas),,https://dthumb-phinf.pstatic.net/?src=%22https...,앤디 워홀과 함께 미국의 팝아트를 대표하는 로이 릭텐스타인(Roy Lichtenst...
9,모나리자,레오나르도 다 빈치,유화,패널에 유채(Huile sur bois),르네상스,https://dthumb-phinf.pstatic.net/?src=%22https...,"15세기경, 패널에 유채로, 크기는 77x53cm이며 프랑스 파리의 루브르 박물관(..."


In [94]:
len(western_data)

3000

In [95]:
western_data = western_data[western_data.text != ""]
western_data.reset_index(drop = True, inplace = True)
western_data.head(20)

,title,artist,art_type,tech,movement,image,text
0,별이 빛나는 밤,빈센트 반 고흐,유화,캔버스에 유채(Oil on canvas),후기인상주의,https://dthumb-phinf.pstatic.net/?src=%22https...,빈센트 반 고흐의 대표작 중 하나로 꼽히는 <별이 빛나는 밤>은 그가 고갱과 다툰 ...
1,아를르의 포룸 광장의 카페 테라스,빈센트 반 고흐,유화,캔버스에 유채(Oil on canvas),후기인상주의,https://dthumb-phinf.pstatic.net/?src=%22https...,"산업 사회로 이행해 가던 19세기 말, 정신적 가치에 대한 열망은 예술가들이 좀 더..."
2,연인(키스),구스타프 클림트,유화,캔버스에 유채(Oil on canvas),상징주의,https://dthumb-phinf.pstatic.net/?src=%22https...,꽃이 흩뿌려진 작은 초원 위에 서 있는 두 연인은 주변과 분리되어 그들을 마치 후광...
3,해바라기,빈센트 반 고흐,유화,캔버스에 유채(Oil on canvas),후기인상주의,https://dthumb-phinf.pstatic.net/?src=%22https...,"1888년, 반 고흐(Vincent Van Gogh, 1853-1890)는 프랑스 ..."
4,아를의 별이 빛나는 밤,빈센트 반 고흐,유화,캔버스에 유채(Huile sur toile),후기인상주의,https://dthumb-phinf.pstatic.net/?src=%22https...,"빈센트 반 고흐가 주고 받았던 서신들을 살펴보면, 그가 얼마나 ‘밤의 효과’에 매료..."
5,에펠탑의 신랑신부,마르크 샤갈,유화,캔버스에 유채(Huile sur toile),,https://dthumb-phinf.pstatic.net/?src=%22https...,말년의 마르크 샤갈이 거주한 라콜린 저택 응접실의 벽난로 위로 그림 한 점이 걸려있...
6,아를의 반 고흐의 방,빈센트 반 고흐,유화,캔버스에 유채(Oil on canvas),후기인상주의,https://dthumb-phinf.pstatic.net/?src=%22https...,반 고흐는 《반 고흐의 방》이라는 이름으로 세 작품을 그렸다. 가장 오래된 버전은 ...
7,모나리자,레오나르도 다 빈치,유화,패널에 유채(Huile sur bois),르네상스,https://dthumb-phinf.pstatic.net/?src=%22https...,"15세기경, 패널에 유채로, 크기는 77x53cm이며 프랑스 파리의 루브르 박물관(..."
8,유디트,구스타프 클림트,유화,캔버스에 유채(Oil on canvas),상징주의,https://dthumb-phinf.pstatic.net/?src=%22https...,"살로메(Salome), 세멜레(Semele), 메데아(Medea), 그리고 유디트(..."
9,행복한 눈물,로이 릭텐스타인,유화,캔버스에 마그나펜(Magna on canvas),,https://dthumb-phinf.pstatic.net/?src=%22https...,앤디 워홀과 함께 미국의 팝아트를 대표하는 로이 릭텐스타인(Roy Lichtenst...


In [146]:
western_data.groupby('artist').text.count().sort_values(ascending=False)[:100]

artist
빈센트 반 고흐                66
클로드 오스카 모네              56
피에르 오귀스트 르누아르           36
구스타프 클림트                25
폴 세잔                    20
폴 고갱                    19
페테르 파울 루벤스              19
산치오 라파엘로                17
자크 루이 다비드               17
에드가 드 가스 일레르 제르망        16
에두아르 마네                 16
베첼리오 티치아노               16
외젠 들라크루아                15
장 오귀스트 도미니크 앵그르         15
르네 마그리트                 15
귀스타브 쿠르베                14
앙리 마티스                  14
레오나르도 다 빈치              13
바실리 칸딘스키                12
프랑수아 부셰                 12
니콜라 푸생                  12
로이 릭텐스타인                11
마르크 샤갈                  10
미켈란젤로 메리시 다 카라바조        10
장 바티스트 카미유 코로           10
앙리 드 툴루즈 로트렉            10
조지프 말로드 윌리엄 터너          10
파블로 피카소                  9
조르주 피에르 쇠라               9
장 프랑수아 밀레                9
                        ..
조르주-앙리 루오                3
장 프레데릭 바지유               3
오노레 도미에                  3
장 앙투안 와토                 3
류명렬                      3
소 한스 홀바인             

---

In [98]:
analyzeDoc = namedtuple('AnalyzedDocument', 'words tags')
type(analyzeDoc)

docs = []

for i in tqdm(range(len(western_data['text']))):
    temp = str(western_data['text'][i])
    words = twitter.morphs(temp)
    tags = [i]
    docs.append(analyzeDoc(words, tags))

100%|██████████| 1021/1021 [00:21<00:00, 47.57it/s]


In [99]:
max_epochs = 20
vec_size = 200
alpha = 0.025

western_model = doc2vec.Doc2Vec(vector_size=vec_size,
                alpha=alpha, 
                min_alpha=0.00025,
                min_count=1,
                dm =1)

western_model.build_vocab(docs)

In [100]:
for epoch in range(max_epochs):
    print('iteration {0}'.format(epoch))
    western_model.train(docs,
                total_examples = western_model.corpus_count,
                epochs=western_model.iter)
    
    # decrease the learning rate
    western_model.alpha -= 0.0002
    
    # fix the learning rate, no decay
    western_model.min_alpha = western_model.alpha

iteration 0


/anaconda3/envs/pythonStudy/lib/python3.6/site-packages/ipykernel_launcher.py:5: DeprecationWarning: Call to deprecated `iter` (Attribute will be removed in 4.0.0, use self.epochs instead).
  """


iteration 1
iteration 2
iteration 3
iteration 4
iteration 5
iteration 6
iteration 7
iteration 8
iteration 9
iteration 10
iteration 11
iteration 12
iteration 13
iteration 14
iteration 15
iteration 16
iteration 17
iteration 18
iteration 19


In [101]:
test_data = twitter.morphs(western_data['text'][0])
test_data

['빈센트',
 '반',
 '고흐',
 '의',
 '대표',
 '작',
 '중',
 '하나로',
 '꼽히는',
 '<',
 '별',
 '이',
 '빛나는',
 '밤',
 '>',
 '은',
 '그',
 '가',
 '고갱',
 '과',
 '다툰',
 '뒤',
 '자신',
 '의',
 '귀',
 '를',
 '자른',
 '사건',
 '이후',
 '생',
 '레미',
 '의',
 '요양원',
 '에',
 '있',
 '을',
 '때',
 '그린',
 '것',
 '이다',
 '.',
 '반',
 '고흐',
 '에게',
 '밤하늘',
 '은',
 '무한',
 '함',
 '을',
 '표현하는',
 '대상',
 '이었',
 '고',
 ',',
 '이보',
 '다',
 '먼저',
 '제작',
 '된',
 '아',
 '를',
 '의',
 '<',
 '밤',
 '의',
 '카페',
 '테라스',
 '>',
 '나',
 '<',
 '론',
 '강',
 '위로',
 '별',
 '이',
 '빛나는',
 '밤',
 '>',
 '에서도',
 '별',
 '이',
 '반짝이는',
 '밤',
 '의',
 '정경',
 '을',
 '다루었',
 '다',
 '.',
 '반',
 '고흐',
 '는',
 '동생',
 '테오',
 '에게',
 '쓴',
 '편지',
 '에서',
 '“',
 '오늘',
 '아침',
 '나',
 '는',
 '해',
 '가',
 '뜨',
 '기',
 '한참',
 '전',
 '에',
 '창문',
 '을',
 '통해',
 '아',
 '무것',
 '도',
 '없고',
 '아주',
 '커',
 '보이는',
 '샛별',
 '밖에',
 '없는',
 '시골',
 '을',
 '보았',
 '다',
 '.',
 '”',
 '고',
 '했',
 '다',
 '.',
 '이',
 '샛별',
 '은',
 '그림',
 '가운데',
 '왼쪽',
 '에',
 '있는',
 '커다란',
 '흰',
 '별일',
 '것',
 '이다',
 '.',
 '그',
 '가',
 '그린',
 '밤하늘',
 '에서는',
 '구름

In [102]:
new_vector = western_model.infer_vector(test_data)
sims = western_model.docvecs.most_similar([new_vector])
sims

[(0, 0.7405378222465515),
 (49, 0.693678617477417),
 (587, 0.6903715133666992),
 (429, 0.6894614696502686),
 (69, 0.6861151456832886),
 (489, 0.6854865550994873),
 (891, 0.6844644546508789),
 (379, 0.6844357252120972),
 (276, 0.6816202402114868),
 (688, 0.6737287044525146)]

In [105]:
western_data.iloc[429]

title                                                  즉흥 III
artist                                               바실리 칸딘스키
art_type                                                   유화
tech                                 캔버스에 유채(Huile sur toile)
movement                                                     
image       https://dthumb-phinf.pstatic.net/?src=%22https...
text        캔버스에 유채로, 크기는 94x130cm이며, 프랑스 파리 조르주 퐁피두센터 국립현...
Name: 429, dtype: object

---

## 빈센트 반 고흐

In [106]:
gogh_data = western_data[western_data['artist'] == '빈센트 반 고흐']

In [110]:
gogh_data.reset_index(drop = True, inplace = True)

In [111]:
analyzeDoc = namedtuple('AnalyzedDocument', 'words tags')
type(analyzeDoc)

type

In [112]:
docs = []

for i in tqdm(range(len(gogh_data['text']))):
    temp = str(gogh_data['text'][i])
    words = twitter.morphs(temp)
    tags = [i]
    docs.append(analyzeDoc(words, tags))

docs

100%|██████████| 66/66 [00:01<00:00, 39.60it/s]


[AnalyzedDocument(words=['빈센트', '반', '고흐', '의', '대표', '작', '중', '하나로', '꼽히는', '<', '별', '이', '빛나는', '밤', '>', '은', '그', '가', '고갱', '과', '다툰', '뒤', '자신', '의', '귀', '를', '자른', '사건', '이후', '생', '레미', '의', '요양원', '에', '있', '을', '때', '그린', '것', '이다', '.', '반', '고흐', '에게', '밤하늘', '은', '무한', '함', '을', '표현하는', '대상', '이었', '고', ',', '이보', '다', '먼저', '제작', '된', '아', '를', '의', '<', '밤', '의', '카페', '테라스', '>', '나', '<', '론', '강', '위로', '별', '이', '빛나는', '밤', '>', '에서도', '별', '이', '반짝이는', '밤', '의', '정경', '을', '다루었', '다', '.', '반', '고흐', '는', '동생', '테오', '에게', '쓴', '편지', '에서', '“', '오늘', '아침', '나', '는', '해', '가', '뜨', '기', '한참', '전', '에', '창문', '을', '통해', '아', '무것', '도', '없고', '아주', '커', '보이는', '샛별', '밖에', '없는', '시골', '을', '보았', '다', '.', '”', '고', '했', '다', '.', '이', '샛별', '은', '그림', '가운데', '왼쪽', '에', '있는', '커다란', '흰', '별일', '것', '이다', '.', '그', '가', '그린', '밤하늘', '에서는', '구름', '과', '대기', ',', '별빛', '과', '달빛', '이', '폭발', '하고', '있다', '.', '황량', '하고', '짙은', '파란색', '하늘', '은', '세상', '의', '종말', '을', '연상', 

In [113]:
# twitter.pos(gogh_data.iloc[0]['text'])

In [114]:
max_epochs = 20
vec_size = 200
alpha = 0.025

gogh_model = doc2vec.Doc2Vec(vector_size=vec_size,
                alpha=alpha, 
                min_alpha=0.00025,
                min_count=1,
                dm =1)

gogh_model.build_vocab(docs)

In [115]:
for epoch in range(max_epochs):
    print('iteration {0}'.format(epoch))
    gogh_model.train(docs,
                total_examples = gogh_model.corpus_count,
                epochs=gogh_model.iter)
    
    # decrease the learning rate
    gogh_model.alpha -= 0.0002
    
    # fix the learning rate, no decay
    gogh_model.min_alpha = gogh_model.alpha

/anaconda3/envs/pythonStudy/lib/python3.6/site-packages/ipykernel_launcher.py:5: DeprecationWarning: Call to deprecated `iter` (Attribute will be removed in 4.0.0, use self.epochs instead).
  """


iteration 0
iteration 1
iteration 2
iteration 3
iteration 4
iteration 5
iteration 6
iteration 7
iteration 8
iteration 9
iteration 10
iteration 11
iteration 12
iteration 13
iteration 14
iteration 15
iteration 16
iteration 17
iteration 18
iteration 19


In [131]:
test_data = twitter.morphs(western_data['text'][3])
new_vector = gogh_model.infer_vector(test_data)
sims = gogh_model.docvecs.most_similar([new_vector])
sims

[(59, 0.7715134620666504),
 (2, 0.7604947686195374),
 (35, 0.7141451239585876),
 (39, 0.7031567096710205),
 (55, 0.6928657293319702),
 (5, 0.6912997961044312),
 (24, 0.6618978977203369),
 (28, 0.6577316522598267),
 (25, 0.653952956199646),
 (65, 0.6235340237617493)]

In [137]:
western_data.iloc[5]

title                                               에펠탑의 신랑신부
artist                                                 마르크 샤갈
art_type                                                   유화
tech                                 캔버스에 유채(Huile sur toile)
movement                                                     
image       https://dthumb-phinf.pstatic.net/?src=%22https...
text        말년의 마르크 샤갈이 거주한 라콜린 저택 응접실의 벽난로 위로 그림 한 점이 걸려있...
Name: 5, dtype: object

---

In [142]:
sample_text = '''시들어가는 1년생 식물들이 허공에 그리는 형상을 나는 사랑한다. 그 형상이 바람과 만나 움직임을 만들어낼 때 그것은 춤과도 같아 보였기에 이 프로젝트를 '허공에 그린 검은 춤'이라 명하였다. 여백은 빛을 품은 하늘이 되고 검은 실루엣은 그들의 존재가 된다. 빛을 등지고 하늘을 배경으로 바라 본 식물들은 고작 들판의 이름 모를 잡초일지언정 나름의 초록빛 르네상스를 지나 황폐하면서도 찬란한 사멸을 묘사하듯 그들의 마지막 제스처는 최고의 구성미를 자아냈다. 시간이 멈춘 듯 바람이 허공에 조각한 듯한 그들의 모습은, 바람이 나부낄 때는 마치 춤을 추는 듯했다. 나는 이 극적인 순간을 표현하고 싶었다.

야생의 시들어가는 1년생 식물은 생생한 다년생 정원수나 꽃집의 풍성하고 화려한 꽃들에서 볼 수 없는, 절묘하게 비틀어지고 뒤틀린 선들을 고스란히 드러낸다. 겨울이 되어 단지 잎만 떨어지는 다년생 식물과 달리 이들은 몸 전체가 다 타버려 재가 되는 느낌이다. 태양에 그을리고 바람에 맞아 우리네 인간이 80년을 살며 몸이라는 옷에 주름을 새기고 떠나듯, 이들의 1년은 우리의 80년을 농축시킨 듯하다. 이러하기에 시들어가는 한해살이 식물이 허공에 그리는 실루엣이 내게는 무엇보다 처절하게 아름다운 공감을 불러 일으킨다.

들판에서 찬바람을 고스란히 맞는 모습이, 생의 마지막 움직임을 허공에 흩날리는 듯한 것이, 내게는 생기발랄하게 보이는 한편 가슴 저미게 애달프기도 하다. 그들의 춤은 땅으로 돌아가기 전 허공에 흩뿌리는 그들의 마지막 역사, 마지막 몸부림으로 다가오는데, 시들어 가는 식물의 뒤틀린 선이 바람에 나부끼는 것이 현대무용에서 무용수들이 관절을 툭툭 쳐서 허공에 내던지듯 그려내는 몸짓과 오버랩된다. 이를 전달하기 위한 노력은 새로운 재료와 새로운 기법의 영역으로 갈 자극제가 되었다. 그리하여 찾은 재료가 바로 자개 (mother of pearl: 진주의 어머니란 뜻을 내포)였다. 자개는 시시각각 변화하여 오묘한 하늘의 빛깔이 되어 식물들의 몸짓을 여백 아닌 여백으로 묘사하기에 더 없이 좋은 매개물이 되었다. 자개 고유의 색은 어떤 컬러 코드로도 포착하기 힘든 홀로그램 빛을 띤다. 분홍빛인 듯 푸른빛이 감돌고 회색 빛 안에서 청록 빛이 반사되는, 단색으로 정의할 수 없는 수많은 얼굴의 하늘 빛깔에 많이 닿아 있다.

반면, 화려한 여백과는 대조적으로 시들어가는 이 식물들의 형상은 검은 색이다. 변화무쌍한 여백과 구분되기 위함이기도 하지만 생멸의 선상 위에 있는 그들의 화려하지 않은 모습을 담기에 매우 적절하다. 왜냐하면 내게 검은 색은 죽음의 색인 동시에 생명의 색을 상징하는데, 숲이 다 타버리고 남은 검은 재를 영양분 삼아 돋아나는 새싹처럼, 소멸이 있은 뒤에야 새로운 시작이 있듯 죽음과 다음 세대로의 순환하는 그 자연의 섭리를 대변하기 때문이다. 검은 색은 그 깊이를 알 수 없는 공간으로 모든 걸 빨아들여서 다른 무언가를 재창조하는 듯하다
'''

In [143]:
print(sample_text)

시들어가는 1년생 식물들이 허공에 그리는 형상을 나는 사랑한다. 그 형상이 바람과 만나 움직임을 만들어낼 때 그것은 춤과도 같아 보였기에 이 프로젝트를 '허공에 그린 검은 춤'이라 명하였다. 여백은 빛을 품은 하늘이 되고 검은 실루엣은 그들의 존재가 된다. 빛을 등지고 하늘을 배경으로 바라 본 식물들은 고작 들판의 이름 모를 잡초일지언정 나름의 초록빛 르네상스를 지나 황폐하면서도 찬란한 사멸을 묘사하듯 그들의 마지막 제스처는 최고의 구성미를 자아냈다. 시간이 멈춘 듯 바람이 허공에 조각한 듯한 그들의 모습은, 바람이 나부낄 때는 마치 춤을 추는 듯했다. 나는 이 극적인 순간을 표현하고 싶었다.

야생의 시들어가는 1년생 식물은 생생한 다년생 정원수나 꽃집의 풍성하고 화려한 꽃들에서 볼 수 없는, 절묘하게 비틀어지고 뒤틀린 선들을 고스란히 드러낸다. 겨울이 되어 단지 잎만 떨어지는 다년생 식물과 달리 이들은 몸 전체가 다 타버려 재가 되는 느낌이다. 태양에 그을리고 바람에 맞아 우리네 인간이 80년을 살며 몸이라는 옷에 주름을 새기고 떠나듯, 이들의 1년은 우리의 80년을 농축시킨 듯하다. 이러하기에 시들어가는 한해살이 식물이 허공에 그리는 실루엣이 내게는 무엇보다 처절하게 아름다운 공감을 불러 일으킨다.

들판에서 찬바람을 고스란히 맞는 모습이, 생의 마지막 움직임을 허공에 흩날리는 듯한 것이, 내게는 생기발랄하게 보이는 한편 가슴 저미게 애달프기도 하다. 그들의 춤은 땅으로 돌아가기 전 허공에 흩뿌리는 그들의 마지막 역사, 마지막 몸부림으로 다가오는데, 시들어 가는 식물의 뒤틀린 선이 바람에 나부끼는 것이 현대무용에서 무용수들이 관절을 툭툭 쳐서 허공에 내던지듯 그려내는 몸짓과 오버랩된다. 이를 전달하기 위한 노력은 새로운 재료와 새로운 기법의 영역으로 갈 자극제가 되었다. 그리하여 찾은 재료가 바로 자개 (mother of pearl: 진주의 어머니란 뜻을 내포)였다. 자개는 시시각각 변화하여 오묘한 하늘의 빛깔이 되어 식물들의 몸짓을 여백 아닌 여백으로 

In [144]:
# 서양화 모델에 적용

test_data = twitter.morphs(sample_text)
new_vector = western_model.infer_vector(test_data)
sims = western_model.docvecs.most_similar([new_vector])
sims

[(236, 0.3217957317829132),
 (521, 0.3197431266307831),
 (570, 0.3196110129356384),
 (42, 0.3173770308494568),
 (124, 0.31149667501449585),
 (686, 0.30603834986686707),
 (672, 0.2911502718925476),
 (77, 0.28993338346481323),
 (590, 0.28925418853759766),
 (276, 0.28564900159835815)]

In [145]:
# 고흐 모델에 적용

test_data = twitter.morphs(sample_text)
new_vector = gogh_model.infer_vector(test_data)
sims = gogh_model.docvecs.most_similar([new_vector])
sims

[(60, 0.2972699999809265),
 (0, 0.26329565048217773),
 (58, 0.26301413774490356),
 (12, 0.24005602300167084),
 (3, 0.2367180585861206),
 (32, 0.22210001945495605),
 (59, 0.2045878767967224),
 (40, 0.18600906431674957),
 (39, 0.17367227375507355),
 (61, 0.16445529460906982)]